In [1]:
import os
import numpy    as np
import pandas   as pd
import psycopg2 as psy

from datetime import datetime as dt
from random   import randrange
from random   import sample
from random   import choice

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
#''' Check if file exists on path. Move existing file to ARCH. Create new input for client_dim.'''


In [3]:
# Setup script input parameters

min_obs = 5                      # minimal number of observation in output dataset
max_obs = 20                     # maximal number of observation in output dataset

min_age = 18                     # minimal age of client
max_age = 90                     # maximal age of client

obs = randrange(min_obs,max_obs) # number of observation
obs

12

In [5]:
main_path    = "Documents/CallCenterStaffing"
path_to_arch = "Arch/client"
path_to_file = "Input"
path_to_dict = "Dict"
file_name    = "client_dim.csv"
name_dict    = "name_dict.csv"
gender_dict  = "gender_dict.csv"

now          = dt.now()

path_to_file = os.path.join(os.path.expanduser("~"),main_path,path_to_file,file_name)
path_to_arch = os.path.join(os.path.expanduser("~"),main_path,path_to_arch)
name_dict    = os.path.join(os.path.expanduser("~"),main_path,path_to_dict,name_dict)
gender_dict  = os.path.join(os.path.expanduser("~"),main_path,path_to_dict,gender_dict)

def is_path_not_correct(path):
    return not(os.path.exists(path))

def is_path_correct(path):
    return os.path.exists(path)

if is_path_not_correct(path_to_file):
    raise Exception("Input file doesn't exist")

if is_path_not_correct(path_to_arch):
    raise Exception("Archive directory doesn't exist")
        
if is_path_not_correct(name_dict):
    raise Exception("Name dictionary doesn't exist")
    
if is_path_not_correct(gender_dict):
    raise Exception("Gender dictionary doesn't exist")
        
time_stamp = "_" + str(now)[:10]
arch_file_name = 'arch_' + file_name.replace(".csv",time_stamp + ".csv")
 
os.rename(path_to_file, os.path.join(path_to_arch, arch_file_name))

Exception: Input file doesn't exist

In [6]:
# connection to database and return list of column column_list
try:
    connection = psy.connect( user = "szymonbocian",
                              password = "",
                              host = "localhost",
                              port = "5432",
                              database = "dwh_call_center")

    cursor = connection.cursor()

    cursor.execute("""
        SELECT column_name 
        FROM INFORMATION_SCHEMA.columns 
        WHERE table_schema = 'stg' AND table_name = 'client_dim';
    """)
    column_list = cursor.fetchall()
    
except (Exception, psy.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
        if(connection):
            cursor.close()
            connection.close()

column_list = [''.join(c for c in s if c not in '(),') for s in column_list]
print(column_list)

['client_first_name', 'client_second_name', 'client_last_name', 'sex_pl', 'sex_eng', 'sex_shortcut_pl', 'sex_shortcut_eng', 'client_age', 'client_business_key']


In [7]:
# Load dictionary with first_name (second_name), last_name
dict_name = pd.read_csv( name_dict,
                         sep = ";",
                         usecols = ['male_name','female_name','last_name'],
                         skiprows = 0
                       )
dict_gender = pd.read_csv( gender_dict,
                           sep = ";",
                           usecols = ['id','sex_pl','sex_eng','sex_shortcut_pl','sex_shortcut_eng'],
                           skiprows = 0,
                           index_col = 'id' 
                         )

In [9]:
f = pd.concat([dict_name.male_name,dict_name.female_name])

df = pd.DataFrame({
    "first_name"  : sample(tuple(f), k = obs),
    "last_name"   : sample(tuple(dict_name.last_name),k = obs),
    "is_sec_name" : np.random.choice([True, False], obs, True, [0.15, 0.85])
})

df['is_male']   = df.first_name.isin(dict_name.male_name)
df['is_female'] = df.first_name.isin(dict_name.female_name)

df['second_name'] = df.apply(lambda l: choice(dict_name.male_name) if l['is_male'] and l['is_sec_name'] else "", axis = 1)
df['second_name'] = df.apply(lambda l: choice(dict_name.female_name) if l['is_female'] and l['is_sec_name'] else l['second_name'], axis = 1)

df = df.merge(dict_gender, 
              left_on  = 'is_male', 
              right_on = 'id', 
              how = 'inner')[['first_name','second_name','last_name','sex_pl','sex_eng','sex_shortcut_pl','sex_shortcut_eng']]

df['client_age'] = pd.Series(np.round(np.random.uniform(min_age,max_age,obs),0)).astype(int)
df

,first_name,second_name,last_name,sex_pl,sex_eng,sex_shortcut_pl,sex_shortcut_eng,client_age
0,Marek,Miłosz,Marcinkiewicz,Mężczyzna,Male,M,M,48
1,Kornel,,Mołdrzyk,Mężczyzna,Male,M,M,51
2,Emilian,,Skrok,Mężczyzna,Male,M,M,51
3,Dobromił,,Siudek,Mężczyzna,Male,M,M,81
4,Jan,Arkadiusz,Hrycyk,Mężczyzna,Male,M,M,18
5,Ludwik,,Waśkowicz,Mężczyzna,Male,M,M,20
6,Michał,,Hejmo,Mężczyzna,Male,M,M,28
7,Teofil,,Rusek,Mężczyzna,Male,M,M,31
8,Ida,,Słomka,Kobieta,Female,K,F,63
9,Edyta,,Bieniasz,Kobieta,Female,K,F,74


In [11]:
# Create client business key 
fnl = ('00' + (df.first_name.str.len() *8).astype(str))
lnl = ('00' + (df.last_name.str.len() *8).astype(str))
cal = ('00' + df.client_age.astype(str))

df['client_business_key'] = df.first_name.map(str).str.slice(0,1) + df.last_name.map(str).str.slice(0,1) + [w[-3:] for w in fnl] + [w[-3:] for w in lnl] + [w[-3:] for w in cal] + df.sex_shortcut_pl

In [12]:
#df = df.drop(['isSecName', 'isMale', 'isFemale'], axis = 1)
df

,first_name,second_name,last_name,sex_pl,sex_eng,sex_shortcut_pl,sex_shortcut_eng,client_age,client_business_key
0,Marek,Miłosz,Marcinkiewicz,Mężczyzna,Male,M,M,48,MM040104048M
1,Kornel,,Mołdrzyk,Mężczyzna,Male,M,M,51,KM048064051M
2,Emilian,,Skrok,Mężczyzna,Male,M,M,51,ES056040051M
3,Dobromił,,Siudek,Mężczyzna,Male,M,M,81,DS064048081M
4,Jan,Arkadiusz,Hrycyk,Mężczyzna,Male,M,M,18,JH024048018M
5,Ludwik,,Waśkowicz,Mężczyzna,Male,M,M,20,LW048072020M
6,Michał,,Hejmo,Mężczyzna,Male,M,M,28,MH048040028M
7,Teofil,,Rusek,Mężczyzna,Male,M,M,31,TR048040031M
8,Ida,,Słomka,Kobieta,Female,K,F,63,IS024048063K
9,Edyta,,Bieniasz,Kobieta,Female,K,F,74,EB040064074K


In [13]:
# create csv file with given destination

df.columns = column_list
df.to_csv(path_to_file, index = None, header = True)